In [ ]:
pip install biolord

In [ ]:
import warnings
import os
import sys
import re
import biolord
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import sklearn
import torch


from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    mutual_info_score
)

In [ ]:
adata = sc.read("/work/Biolord_all/new_data_raw/healthy_hamstring_processed_adata_raw.h5ad")

In [ ]:
adata

In [ ]:
# Split the data set into train and test
from sklearn.model_selection import train_test_split


split_key = "split"
adata.obs[split_key] = "train"
idx = list(range(len(adata)))
idx_train, idx_test = train_test_split(adata.obs_names, test_size=0.1, random_state=42)
adata.obs.loc[idx_train, split_key] = "train"
adata.obs.loc[idx_test, split_key] = "test"

In [ ]:
adata_normal = adata.copy()

In [ ]:
adata_normal

In [ ]:
# Normalizing to median total counts
sc.pp.normalize_total(adata_normal, target_sum=1e4)
# Logarithmize the data
sc.pp.log1p(adata_normal)

In [ ]:
adata

In [ ]:
# Filter the data to use only the training set and make a copy
adata_train = adata_normal[adata_normal.obs[split_key] == "train"].copy()
adata_test = adata_normal[adata_normal.obs[split_key] == "test"].copy()

In [ ]:
biolord.Biolord.setup_anndata(
    adata_normal, ordered_attributes_keys=None, categorical_attributes_keys=["cell_type", "sex", "donor_id"]
)

In [ ]:
# Instantiate a Biolord model

module_params = {
    "decoder_width": 1024,
    "decoder_depth": 4,
    "attribute_nn_width": 512,
    "attribute_nn_depth": 2,
    "n_latent_attribute_categorical": 4,
    "gene_likelihood": "normal",
    "reconstruction_penalty": 1e2,
    "unknown_attribute_penalty": 1e1,
    "unknown_attribute_noise_param": 1e-1,
    "attribute_dropout_rate": 0.1,
    "use_batch_norm": False,
    "use_layer_norm": False,
    "seed": 42,
}

In [ ]:
model = biolord.Biolord(
    adata=adata_normal,
    n_latent=32,
    model_name="Healthy_Hamstring_Biolord_check_with_local_norm",
    module_params=module_params,
    train_classifiers=False,
    split_key="split",
)

In [ ]:
# Train the model
trainer_params = {
    "n_epochs_warmup": 0,
    "latent_lr": 1e-4,
    "latent_wd": 1e-4,
    "decoder_lr": 1e-4,
    "decoder_wd": 1e-4,
    "attribute_nn_lr": 1e-2,
    "attribute_nn_wd": 4e-8,
    "step_size_lr": 45,
    "cosine_scheduler": True,
    "scheduler_final_lr": 1e-5,
}

In [ ]:
model.train(
    max_epochs=500,
    batch_size=512,
    plan_kwargs=trainer_params,
    early_stopping=True,
    early_stopping_patience=20,
    check_val_every_n_epoch=10,
    num_workers=1,
    enable_checkpointing=False,
)

In [ ]:
model.save("/work/Biolord_all/Healthy_Hamstring_Biolord/Newest_Biolord_HH/Biolord_Healthy_Hamstring/HH_Biolord_test_raw_extra")

In [ ]:
model = model.load("/work/Biolord_all/Healthy_Hamstring_Biolord/Newest_Biolord_HH/Biolord_Healthy_Hamstring/HH_Biolord_test_raw_extra_lesscov", adata = adata_normal)

In [ ]:
model

In [ ]:
## Recovery for all the data
z_basal, z = model.get_latent_representation_adata(
    adata_normal,
    batch_size=256,
)
#rec, _ = model.predict(adata_test, batch_size=256)

In [ ]:
z_residual = z[:, z_basal.shape[1]:]

In [ ]:
z_residual

In [ ]:
print("z_basal shape:", z_basal.X.shape) 
print("z shape:", z.X.shape)             

# Unknown: 
z_unknown = z.X[:, z_basal.X.shape[1]:]
print("z_unknown shape:", z_unknown.shape)  

In [ ]:
z

In [ ]:
z.X

In [ ]:
rec, _ = model.predict(adata_normal[adata_normal.obs["split"] == "test"])

In [ ]:
rec.X

In [ ]:
# Reverse the log1p 
rec.X = np.expm1(rec.X)

In [ ]:
true_norm_size = 1e4

In [ ]:
rec.X = (rec.X / true_norm_size) * adata_test.obs["total_counts"].values[:, None]

In [ ]:
adata_test.var["total_counts"]

In [ ]:
# Step 1: Get the raw test data from the original unnormalized adata
X_true = adata[adata.obs["split"] == "test"].X  # From raw (not normalized) data
rec_array = rec.X

In [ ]:

df = adata.obs[["donor_id", "development_stage", "development_stage_ontology_term_id"]].copy()
# Check how many unique combinations exist
unique_combinations = df.drop_duplicates()
print(f"Unique combinations:\n{unique_combinations}")
# Check pairwise uniqueness
for col1 in df.columns:
    for col2 in df.columns:
        if col1 != col2:
            is_unique = df.groupby(col1)[col2].nunique().max() == 1
            print(f"{col1} → {col2} is one-to-one: {is_unique}")


In [ ]:
from sklearn.metrics import r2_score
import scipy.sparse

# Convert to dense
if scipy.sparse.issparse(X_true):
    X_true = X_true.toarray()
if scipy.sparse.issparse(rec_array):
    rec_array = rec_array.toarray()

# Flatten
X_true_flat = X_true.flatten()
rec_flat = rec_array.flatten()

# R² score
r2_flat = r2_score(X_true_flat, rec_flat)
print(" Global R² (flattened):", r2_flat)

In [ ]:
# MSE when test_adata is passed 

mse = mean_squared_error(X_true,rec_array)

print(f"MSE score with test_adata: {mse}")

In [ ]:
# MSE when test_adata is passed 

mae = mean_absolute_error(X_true, rec_array)

print(f"MAE score with test_adata: {mae}")

In [ ]:
# If rec is an AnnData object, extract the X attribute (i.e., the data matrix)
import anndata
if isinstance(rec, anndata.AnnData):
    rec = rec.X

# Now, rec should be a numpy array or sparse matrix, which is what obsm expects
adata_test.obsm["X_reconstructed"] = rec

# Save the entire object with the reconstructed data
adata_test.write("biolord_HH__fix_final_removed_batch_effect.h5ad")

In [ ]:
# Step 2: Save original raw counts
X_original = adata.X.copy()

In [ ]:
# Get original total counts
if scipy.sparse.issparse(X_original):
    total_counts = np.array(X_original.sum(axis=1)).flatten()
else:
    total_counts = X_original.sum(axis=1)

In [ ]:
# Normalize and log1p
adata_tmp = adata.copy()
sc.pp.normalize_total(adata_tmp, target_sum=1e4)
sc.pp.log1p(adata_tmp)
X_norm_log = adata_tmp.X.copy()

In [ ]:
# Reverse log1p
if scipy.sparse.issparse(X_norm_log):
    X_reversed = np.expm1(X_norm_log.toarray())
else:
    X_reversed = np.expm1(X_norm_log)

In [ ]:
# Reverse normalization
X_reconstructed = (X_reversed / 1e4) * total_counts[:, None]

In [ ]:
# Convert original to dense for comparison
if scipy.sparse.issparse(X_original):
    X_original_dense = X_original.toarray()
else:
    X_original_dense = X_original


In [ ]:
# Compute error
mae = np.mean(np.abs(X_reconstructed - X_original_dense))
max_diff = np.max(np.abs(X_reconstructed - X_original_dense))

print(f"Mean Absolute Error (round-trip): {mae:.4f}")

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.cluster import KMeans
from sklearn.metrics import mutual_info_score
from sklearn.model_selection import train_test_split
###MIG###
import numpy as np
import anndata
import pandas as pd

def encode_categorical(data):
    encoders = []
    encoded_data = np.zeros_like(data, dtype=int)
    for i in range(data.shape[1]):
        le = LabelEncoder()
        encoded_data[:, i] = le.fit_transform(data[:, i])
        encoders.append(le)
    return encoded_data, encoders

def prep_data(adata, embedding, covriate_keys=None):
    encoded_factors_of_variation, _ = encode_categorical(adata.obs[covriate_keys].values)

    if isinstance(embedding, anndata.AnnData):  
        embedding_data = embedding.X
    else:
        embedding_data = embedding

    mus = np.array(embedding_data)
    ys = np.array(encoded_factors_of_variation)

    return mus.T.copy(), ys.T.copy()


def compute_mig(mus, ys, covariate_names=None):
    """Computes the mutual information gap."""
    return _compute_mig(mus, ys, covariate_names)

def _compute_mig(mus, ys, covariate_names=None):
    """Computes MIG score based on latent codes and covariates."""
    score_dict = {}
    discretized_mus = make_discretizer(mus, discretizer_fn=_histogram_discretize)
   # print("Sample Discretized Latent Variables:\n", discretized_mus[:, :5])
    m = discrete_mutual_info(discretized_mus, ys)

    if covariate_names is None:
        covariate_names = [f"Covariate {j}" for j in range(m.shape[1])]
        
    for j in range(m.shape[1]):
        top_indices = np.argsort(m[:, j])[::-1][:3]
        top_scores = m[top_indices, j]
        print(f"Top 3 MI scores for covariate '{covariate_names[j]}':")
        for idx, score in zip(top_indices, top_scores):
            print(f"  Latent dim {idx}: MI = {score:.4f}")

    assert m.shape[0] == mus.shape[0]
    assert m.shape[1] == ys.shape[0]

    entropy = discrete_entropy(ys)
    sorted_m = np.sort(m, axis=0)[::-1]

    score_dict["discrete_mig"] = np.mean(
        np.divide(sorted_m[0, :] - sorted_m[1, :], entropy[:])
    )

    print("Þetta er score:", score_dict)
    print("Entropy values:", entropy)
    return score_dict

def discrete_mutual_info(mus, ys):
    num_codes = mus.shape[0]
    num_factors = ys.shape[0]
    m = np.zeros([num_codes, num_factors])
    
    for i in range(num_codes):
        for j in range(num_factors):
            m[i, j] = mutual_info_score(ys[j, :], mus[i, :])
    
    return m

def discrete_entropy(ys):
    num_factors = ys.shape[0]
    h = np.zeros(num_factors)
    
    for j in range(num_factors):
        h[j] = mutual_info_score(ys[j, :], ys[j, :])
    
    return h

def _identity_discretizer(target, num_bins):
    del num_bins
    return target


def _histogram_discretize(target, num_bins=10):
    discretized = np.zeros_like(target)
    for i in range(target.shape[0]):
        discretized[i, :] = np.digitize(target[i, :], np.histogram(
            target[i, :], num_bins)[1][:-1])
    return discretized


def make_discretizer(target, num_bins=10, discretizer_fn=_histogram_discretize):
    return discretizer_fn(target, num_bins)


def score_disentanglement(adata, embedding_data, embedding_basal, covriate_keys=None, continuous_covriate_keys=None):
    mus, ys = prep_data(adata, embedding_data, covriate_keys=covriate_keys)
    print('Computing MIG')
    mig = compute_mig(mus, ys, covariate_names=covriate_keys)
    return mig, mus, ys

# Run MIG score
mig_1,mus,ys = score_disentanglement(
    adata_normal,
    z,
    None,
    covriate_keys=["cell_type", "sex", "donor_id", "development_stage_ontology_term_id", "development_stage"]
)

print("MIG Score:", mig_1)
print("Latent variances:", np.var(mus, axis=1))

discretized_mus = _histogram_discretize(mus)
m = discrete_mutual_info(discretized_mus, ys)

print("MI matrix shape:", m.shape)
print("Max MI per factor:", np.max(m, axis=0))
print("Which latents have highest MI per factor:", np.argmax(m, axis=0))




In [ ]:
covriate_keys = ["cell_type", "sex", "donor_id", "development_stage_ontology_term_id", "development_stage"]
for key in covriate_keys:
    print(f"{key} — unique values:")
    print(adata_normal.obs[key].unique())
    print("=" * 40)


In [ ]:
# Finalized DCI computation based on disentanglement_lib

import numpy as np
import pandas as pd
import anndata
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import entropy

# === Encoding and Preprocessing ===
def encode_categorical(data):
    encoded_data = np.zeros_like(data, dtype=int)
    for i in range(data.shape[1]):
        le = LabelEncoder()
        encoded_data[:, i] = le.fit_transform(data[:, i])
    return encoded_data

def remove_duplicate_columns(df):
    df_unique = df.T.drop_duplicates().T
    return df_unique

def prep_data(adata, embedding, covariate_keys, test_size=0.25):
    idx_train, idx_test = train_test_split(
        range(len(adata)), test_size=test_size, random_state=42
    )
    cov_df = adata.obs[covariate_keys].copy()
    cov_df = remove_duplicate_columns(cov_df)
    encoded_factors = encode_categorical(cov_df.values)
    embedding_data = embedding.X if isinstance(embedding, anndata.AnnData) else embedding
    mus_train = embedding_data[idx_train]
    mus_test = embedding_data[idx_test]
    ys_train = encoded_factors[idx_train]
    ys_test = encoded_factors[idx_test]
    return mus_train.T, ys_train.T, mus_test.T, ys_test.T

# === Importance Matrix ===
def compute_importance_rf(x_train, y_train, x_test, y_test):
    num_factors = y_train.shape[0]
    num_codes = x_train.shape[0]
    importance_matrix = np.zeros((num_codes, num_factors))
    train_acc = []
    test_acc = []
    for i in range(num_factors):
        model = RandomForestClassifier(random_state=42, max_depth=5)
        model.fit(x_train.T, y_train[i])
        importance_matrix[:, i] = np.abs(model.feature_importances_)
        train_acc.append(np.mean(model.predict(x_train.T) == y_train[i]))
        test_acc.append(np.mean(model.predict(x_test.T) == y_test[i]))
    return importance_matrix, np.mean(train_acc), np.mean(test_acc)

# === Disentanglement ===
def disentanglement_per_code(importance_matrix):
    row_sums = importance_matrix.sum(axis=1, keepdims=True)
    safe_matrix = np.where(row_sums == 0, 1e-11, row_sums)
    normalized = importance_matrix / safe_matrix
    return 1. - entropy(normalized.T + 1e-11, base=importance_matrix.shape[1])

def disentanglement(importance_matrix):
    per_code = disentanglement_per_code(importance_matrix)
    total = importance_matrix.sum()
    if total == 0.:
        return 0.0
    code_importance = importance_matrix.sum(axis=1) / total
    return np.sum(per_code * code_importance)

# === Completeness ===
def completeness_per_factor(importance_matrix):
    return 1. - entropy(importance_matrix + 1e-11, base=importance_matrix.shape[0])

def completeness(importance_matrix):
    per_factor = completeness_per_factor(importance_matrix)
    total = importance_matrix.sum()
    if total == 0.:
        return 0.0
    factor_importance = importance_matrix.sum(axis=0) / total
    return np.sum(per_factor * factor_importance)

# === DCI Master Function ===
def compute_dci(mus_train, ys_train, mus_test, ys_test):
    importance_matrix, train_acc, test_acc = compute_importance_rf(
        mus_train, ys_train, mus_test, ys_test
    )
    threshold = 1e-11
    importance_matrix = np.where(importance_matrix < threshold, 0, importance_matrix)
    return {
    "disentanglement": disentanglement(importance_matrix),
    "completeness": completeness(importance_matrix),
    "importance_matrix": importance_matrix,
    "train_acc": train_acc,
    "test_acc": test_acc,
}


In [ ]:
covariate_keys = ["cell_type", "sex", "donor_id"]
mus_train, ys_train, mus_test, ys_test = prep_data(adata, z, covariate_keys)
dci_scores = compute_dci(mus_train, ys_train, mus_test, ys_test)
print(dci_scores)

importance_matrix = dci_scores["importance_matrix"]

In [ ]:
import numpy as np
from sklearn import svm
import logging

logging.basicConfig(level=logging.INFO)

def compute_sap(mus, ys, mus_test, ys_test, continuous_factors):
    """
    Computes the SAP score.

    Args:
        mus: (latent_dim, num_samples) training representations
        ys: (num_factors, num_samples) training factors
        mus_test: (latent_dim, num_samples) test representations
        ys_test: (num_factors, num_samples) test factors
        continuous_factors: Whether the factors are continuous

    Returns:
        Dictionary with SAP score.
    """
    logging.info("Computing SAP score...")
    _validate_shapes(mus, ys, mus_test, ys_test)

    score_matrix = compute_score_matrix(mus, ys, mus_test, ys_test, continuous_factors)
    scores_dict = {}
    scores_dict["SAP_score"] = compute_avg_diff_top_two(score_matrix)
    logging.info("SAP score: %.4f", scores_dict["SAP_score"])
    return scores_dicte


def compute_score_matrix(mus, ys, mus_test, ys_test, continuous_factors):
    """Compute score matrix for each latent and factor combination."""
    num_latents, num_samples = mus.shape
    num_factors = ys.shape[0]

    score_matrix = np.zeros([num_latents, num_factors])
    for i in range(num_latents):
        for j in range(num_factors):
            mu_i = mus[i, :]
            y_j = ys[j, :]

            if continuous_factors:
                cov_mu_i_y_j = np.cov(mu_i, y_j, ddof=1)
                cov_mu_y = cov_mu_i_y_j[0, 1] ** 2
                var_mu = cov_mu_i_y_j[0, 0]
                var_y = cov_mu_i_y_j[1, 1]
                if var_mu > 1e-12 and var_y > 1e-12:
                    score_matrix[i, j] = cov_mu_y / (var_mu * var_y)
                else:
                    score_matrix[i, j] = 0.0
            else:
                mu_i_test = mus_test[i, :]
                y_j_test = ys_test[j, :]
                classifier = svm.LinearSVC(C=0.01, class_weight="balanced", max_iter=10000)
                classifier.fit(mu_i[:, np.newaxis], y_j)
                pred = classifier.predict(mu_i_test[:, np.newaxis])
                score_matrix[i, j] = np.mean(pred == y_j_test)

    return score_matrix


def compute_avg_diff_top_two(score_matrix):
    """Compute average difference between top two latent scores per factor."""
    sorted_matrix = np.sort(score_matrix, axis=0)
    return np.mean(sorted_matrix[-1, :] - sorted_matrix[-2, :])


def _validate_shapes(mus, ys, mus_test, ys_test):
    """Ensure shapes are valid for SAP computation."""
    assert mus.shape[1] == ys.shape[1], "Mismatch: mus and ys sample count"
    assert mus_test.shape[1] == ys_test.shape[1], "Mismatch: mus_test and ys_test sample count"
    assert mus.shape[0] == mus_test.shape[0], "Mismatch in latent dimensions"
    assert ys.shape[0] == ys_test.shape[0], "Mismatch in number of factors"



# mus, ys: (latent_dim, num_samples)
# mus_test, ys_test: same shapes
sap_score = compute_sap(mus_train, ys_train, mus_test, ys_test, continuous_factors=False)
print("SAP:", sap_score["SAP_score"])


import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def plot_sap_score_matrix(score_matrix, factor_names=None, latent_names=None, title="SAP Score Matrix"):
    plt.figure(figsize=(10, 6))
    sns.heatmap(score_matrix, annot=True, fmt=".2f", cmap="viridis", cbar_kws={"label": "Score"})
    
    plt.xlabel("Factors")
    plt.ylabel("Latent Dimensions")
    if factor_names:
        plt.xticks(ticks=np.arange(len(factor_names)) + 0.5, labels=factor_names, rotation=45, ha="right")
    if latent_names:
        plt.yticks(ticks=np.arange(len(latent_names)) + 0.5, labels=latent_names, rotation=0)
    
    plt.title(title)
    plt.tight_layout()
    plt.show()
score_matrix = compute_score_matrix(mus_train, ys_train, mus_test, ys_test, continuous_factors=False)
plot_sap_score_matrix(score_matrix)



In [ ]:
# IRS 


def compute_irs(mus, ys, diff_quantile=0.99):
    ys_discrete = make_discretizer(ys)

    active_mask = (mus.var(axis=1) > 0)
    active_mus = mus[active_mask, :]

    if active_mus.size == 0:
        irs_score = 0.0
    else:
        irs_score = scalable_disentanglement_score(ys_discrete.T, active_mus.T, diff_quantile)["avg_score"]

    score_dict = {}
    score_dict["IRS"] = irs_score
    score_dict["num_active_dims"] = int(np.sum(active_mask))
    return score_dict


def _drop_constant_dims(ys):
    """Returns a view of the matrix `ys` with dropped constant rows."""
    ys = np.asarray(ys)
    if ys.ndim != 2:
        raise ValueError("Expecting a matrix.")

    variances = ys.var(axis=1)
    active_mask = variances > 0.
    return ys[active_mask, :]


def scalable_disentanglement_score(gen_factors, latents, diff_quantile=0.99):
    """Computes IRS scores of a dataset.

    Assumes no noise in X and crossed generative factors (i.e. one sample per
    combination of gen_factors). Assumes each g_i is an equally probable
    realization of g_i and all g_i are independent.

    Args:
        gen_factors: Numpy array of shape (num samples, num generative factors),
            matrix of ground truth generative factors.
        latents: Numpy array of shape (num samples, num latent dimensions), matrix
            of latent variables.
        diff_quantile: Float value between 0 and 1 to decide what quantile of diffs
            to select (use 1.0 for the version in the paper).

    Returns:
        Dictionary with IRS scores.
    """
    num_gen = gen_factors.shape[1]
    num_lat = latents.shape[1]

    # Compute normalizer.
    max_deviations = np.max(np.abs(latents - latents.mean(axis=0)), axis=0)
    cum_deviations = np.zeros([num_lat, num_gen])
    for i in range(num_gen):
        unique_factors = np.unique(gen_factors[:, i], axis=0)
        assert unique_factors.ndim == 1
        num_distinct_factors = unique_factors.shape[0]
        for k in range(num_distinct_factors):
            # Compute E[Z | g_i].
            match = gen_factors[:, i] == unique_factors[k]
            e_loc = np.mean(latents[match, :], axis=0)

            # Difference of each value within that group of constant g_i to its mean.
            diffs = np.abs(latents[match, :] - e_loc)
            max_diffs = np.percentile(diffs, q=diff_quantile*100, axis=0)
            cum_deviations[:, i] += max_diffs
        cum_deviations[:, i] /= num_distinct_factors
    # Normalize value of each latent dimension with its maximal deviation.
    normalized_deviations = cum_deviations / max_deviations[:, np.newaxis]
    irs_matrix = 1.0 - normalized_deviations
    disentanglement_scores = irs_matrix.max(axis=1)
    if np.sum(max_deviations) > 0.0:
        avg_score = np.average(disentanglement_scores, weights=max_deviations)
    else:
        avg_score = np.mean(disentanglement_scores)

    parents = irs_matrix.argmax(axis=1)
    score_dict = {}
    score_dict["disentanglement_scores"] = disentanglement_scores
    score_dict["avg_score"] = avg_score
    score_dict["parents"] = parents
    score_dict["IRS_matrix"] = irs_matrix
    score_dict["max_deviations"] = max_deviations
    return score_dict



irs = compute_irs(mus_train, ys_train, diff_quantile=0.99)
irs

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.heatmap(importance_matrix, cmap="viridis", annot=False)
plt.xlabel("Factors")
plt.ylabel("Latent dimensions")
plt.title("DCI Importance Matrix")
plt.tight_layout()
plt.show()